<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/Best_Match_Society_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from functools import reduce
import time
from geopy.distance import geodesic
from google.colab import drive, auth
from oauth2client.client import GoogleCredentials
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

def changeDateFormat(row):
    return pd.to_datetime(row, errors='coerce').dt.date

def cleanText(row):
    return row.str.upper().str.strip()

def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows

def gdata(spreadSheetId, rangeName):
    l = get_data(spreadSheetId, rangeName)
    ldf = pd.DataFrame.from_records(l[1:], columns=l[0])
    return ldf

def sdata(spreadsheet_id, sheet_name, dataframe, start_row=1, start_col=1):
    wb = gc.open_by_key(spreadsheet_id)
    ws = wb.worksheet(sheet_name)
    range_to_clear = f"{sheet_name}!A{start_row}:ZZZ"
    wb.values_clear(range_to_clear)
    set_with_dataframe(ws, dataframe, row=start_row, col=start_col)

def clean_numeric_columns(df, columns):
    for column in columns:
        # Remove non-numeric characters
        df[column] = df[column].str.extract(r'([0-9\.\-]+)').astype(float)
    return df

daysDelta = 0
DumpDate =  pd.to_datetime((datetime.today() - timedelta(days = daysDelta)).date()).strftime('%d %B %Y')
file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"+(datetime.today() - timedelta(days = daysDelta)).strftime("%b %Y")+"/"

sm=gdata("1aVAu23rPNHyOEk2ioNXdbqyUKEFzcXedlY283cvDKvY","Society Master")
sm=sm[sm['Society status']=='ACTIVE']
sm=sm[['Kibana ID','Society Name']]
sm.rename(columns={'Kibana ID':'id'},inplace=True)
dump_name = "Society Creation Date - Insights Copy"

scd = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)
scd=scd[['id','latitude', 'longitude']]

scd['latitude'] = scd['latitude'].apply(lambda x: float(x.replace(',', '')) if isinstance(x, str) else x)
scd['longitude'] = scd['longitude'].apply(lambda x: float(x.replace(',', '')) if isinstance(x, str) else x)

scd[['latitude', 'longitude']] = scd[['latitude', 'longitude']]/1000000

sm=sm.merge(scd,on='id',how='left')
sm.drop_duplicates(keep='first',inplace=True)
sm=sm[(sm['latitude'].notna()) & (sm['longitude'].notna())]

js=gdata("1y8dDnmbFl2IInlhzeCLZq9XhPoSL6VjF5F2MtihAudo","Store Data")
js=js[['Store Code','Store_Latitude', 'Store_Longitude','Enter The Distance Limit']]

# Clean the columns
js = clean_numeric_columns(js, ['Store_Latitude', 'Store_Longitude', 'Enter The Distance Limit'])

js[['Store_Latitude', 'Store_Longitude','Enter The Distance Limit']]=js[['Store_Latitude', 'Store_Longitude','Enter The Distance Limit']].astype(float)
js['Remark'] = 'Invalid'
js.loc[(js['Store_Latitude'] >= 8.4) & (js['Store_Latitude'] <= 37.6) &
       (js['Store_Longitude'] >= 68.1) & (js['Store_Longitude'] <= 97.4), 'Remark'] = 'Valid'
js=js[js['Remark']=='Valid']

def calculate_distance(row):
    society_coords = (row['latitude'], row['longitude'])
    store_coords = (row['Store_Latitude'], row['Store_Longitude'])
    return geodesic(society_coords, store_coords).kilometers

# Assuming df1 is the society dataframe and df2 is the store dataframe
result = pd.merge(sm, js, how='cross')

# Split 'df1' into three parts
total_size = len(result)
part_size = total_size // 3
df11 = result.iloc[:part_size]
df12 = result.iloc[part_size:2*part_size]
df13 = result.iloc[2*part_size:]
#1st part
st=time.time()
df11['distance_km'] = df11.apply(lambda row: geodesic((row['latitude'], row['longitude']), (row['Store_Latitude'], row['Store_Longitude'])).kilometers, axis=1)
et=time.time()

elapsed_time_minutes = (et - st) / 60

# Print the elapsed time in minutes
print(f"Elapsed time: {elapsed_time_minutes:.4f} minutes")

#2nd part
st=time.time()
df12['distance_km'] = df12.apply(lambda row: geodesic((row['latitude'], row['longitude']), (row['Store_Latitude'], row['Store_Longitude'])).kilometers, axis=1)
et=time.time()

elapsed_time_minutes = (et - st) / 60

# Print the elapsed time in minutes
print(f"Elapsed time: {elapsed_time_minutes:.4f} minutes")

#3rd part
st=time.time()
df13['distance_km'] = df13.apply(lambda row: geodesic((row['latitude'], row['longitude']), (row['Store_Latitude'], row['Store_Longitude'])).kilometers, axis=1)
et=time.time()

elapsed_time_minutes = (et - st) / 60

# Print the elapsed time in minutes
print(f"Elapsed time: {elapsed_time_minutes:.4f} minutes")
pdata=pd.concat([df11,df12,df13])
pdata=pdata[(pdata['distance_km'] <= pdata['Enter The Distance Limit'])].sort_values(['id','distance_km'])
pdata_s=pdata.drop_duplicates('id',keep='first')
final_cata=pdata_s[['Store Code','id','Society Name','distance_km']]
final_cata.rename(columns={'id':'Society ID','distance_km':'Distance Between Store & Society'},inplace=True)
final_cata=final_cata[['Store Code','Society ID','Society Name','Distance Between Store & Society']]

l = get_data("1YmTPZYOv1iP6jwnL_kGEchHEv7TkhZIh0c5HBp63lR4", 'Helper')
df2 = pd.DataFrame.from_records(l[1:], columns=l[0])

l = get_data("1vVgJJdS5-qu9ga6Hag0WRuSSAUGI1WrWnWsgRRMir_g", 'Helper')
prm = pd.DataFrame.from_records(l[1:], columns=l[0])
prm=prm[['Kibana ID','Premiumness']]
df2=pd.merge(df2,prm,on='Kibana ID',how='left')

df2=df2[['City', 'Kibana ID', 'Original Closure Number',
       'Society status', 'Live Product', 'Latest Signing Date', 'Sold Flats',
       'KAM Usage', 'Sr.KAM Usage', 'Download %',
       'Open to Sales Status-ERP', 'Open to Sales Status-VMS',
       'Status', 'Rent Range',
       'Locality (Automated Through Latlong)', 'Premiumness', 'pincode',
       'Locality', 'fashion quartile', 'food quartile', 'grocery quartile',
       'cabs quartile', 'ecom quartile', 'female to male ratio',
       'Pet Quartiles', 'Owner Residing penetration',
       'No: of Activity done in last  6 months']]

df2 = df2.loc[:, ~df2.columns.duplicated()]



'''df2=df2[['City', 'Kibana ID', 'Original Closure Number',
       'societyStatus', 'Live Product', 'Latest Signing Date', 'Sold Flats',
       'KAM Usage', 'Sr.KAM Usage', 'Download %',
       'Open to Sales Status-ERP', 'Open to Sales Status-VMS',
       'Status', 'latitude', 'longitude', 'Rent Range',
       'Locality (Automated Through Latlong)', 'Premiumness', 'pincode',
       'Locality', 'fashion quartile', 'food quartile', 'grocery quartile',
       'cabs quartile', 'ecom quartile', 'female to male ratio',
       'Pet Quartiles', 'Owner Residing penetration',
       'No: of Activity done in last  6 months']]'''
df2.rename(columns={'Kibana ID':'Society ID'},inplace=True)
final_cata=final_cata.merge(df2,on='Society ID',how='left')



#Pushing to sheet
spreadsheet_id='1y8dDnmbFl2IInlhzeCLZq9XhPoSL6VjF5F2MtihAudo'
sheet_name= "Best Match Result"
start_row=1
start_col=1
wb = gc.open_by_key(spreadsheet_id)
ws = wb.worksheet(sheet_name)
range_to_clear = f"{sheet_name}!A{start_row}:AM"
wb.values_clear(range_to_clear)
set_with_dataframe(ws, final_cata, row=start_row, col=start_col)

Mounted at /content/drive


<ipython-input-1-442248655c4c>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['distance_km'] = df11.apply(lambda row: geodesic((row['latitude'], row['longitude']), (row['Store_Latitude'], row['Store_Longitude'])).kilometers, axis=1)


Elapsed time: 4.0130 minutes


<ipython-input-1-442248655c4c>:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12['distance_km'] = df12.apply(lambda row: geodesic((row['latitude'], row['longitude']), (row['Store_Latitude'], row['Store_Longitude'])).kilometers, axis=1)


Elapsed time: 3.9543 minutes


<ipython-input-1-442248655c4c>:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df13['distance_km'] = df13.apply(lambda row: geodesic((row['latitude'], row['longitude']), (row['Store_Latitude'], row['Store_Longitude'])).kilometers, axis=1)


Elapsed time: 3.9808 minutes


<ipython-input-1-442248655c4c>:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_cata.rename(columns={'id':'Society ID','distance_km':'Distance Between Store & Society'},inplace=True)
